<a href="https://colab.research.google.com/github/Redcof/Agendect/blob/master/Advance_Telecom_Churn_Assgnment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Soumen : Jan-16-2022  : Basic setup 1hrs
# Soumen : Jan-17-2022  : Understainding problem statement 1hrs
# Soumen : Jan-19-2022 21:08-23:23 : understanding the problem and datatype fixing
# Soumen : Jan-20-2022 10:53-11:42 Understanding the problem statement
# Soumen : Jan-20-2022 22:50-00:42 Understanding the problem statement and missing value handeling
# Soumen : Jan-21-2022 10:30-12:52 Data processing for high value customer and google colab support


In [ ]:
# download training and testing data
import requests

def download_file_from_google_drive(url, destination):
    """Download form google drive"""
    id = url.split('/')[-2]

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    print(f"Downloading...{url}")
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    if "Sign in to continue to Google Drive" in str(response.text):
        print("Please create a sharable link with 'Any one with the link' and 'Viewer' previledge.")
        print(f"Download failed-{url}")
        return
    save_response_content(response, destination)    

def get_confirm_token(response):
    """Drive authentication with token"""
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    """Save response data to a file"""
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    print(f"Saved...{destination}")

In [3]:
# set it False if running locally
drive = True

if drive:
    urls = [
            ("https://drive.google.com/file/d/1As9_K9Fc-huoy9k6NJUFj0N7r8_FgAhY/view?usp=sharing", "telecom_churn_data.csv"), 
            ("https://docs.google.com/spreadsheets/d/1ArF93y3F1FYlv5YKH54TSM3EYmuT4CAz/edit?usp=sharing&ouid=110160683155830172199&rtpof=true&sd=true", "Data+Dictionary-+Telecom+Churn+Case+Study.xlsx"),
            ("https://drive.google.com/file/d/10ZiQvIjLzerzufRCV91ed76otx6x5ymQ/view?usp=sharing", "common.ipynb")
    ]
    for url, path in urls:
        download_file_from_google_drive(url, f"/content/{path}")


Downloading...https://drive.google.com/file/d/1As9_K9Fc-huoy9k6NJUFj0N7r8_FgAhY/view?usp=sharing
Saved.../content/telecom_churn_data.csv
Downloading...https://docs.google.com/spreadsheets/d/1ArF93y3F1FYlv5YKH54TSM3EYmuT4CAz/edit?usp=sharing&ouid=110160683155830172199&rtpof=true&sd=true
Saved.../content/Data+Dictionary-+Telecom+Churn+Case+Study.xlsx
Downloading...https://drive.google.com/file/d/10ZiQvIjLzerzufRCV91ed76otx6x5ymQ/view?usp=sharing
Saved.../content/common.ipynb


In [4]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", -1)

# Problem Statement

## Business problem overview

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business goal.
 
To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.
 
In this project, you will analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.
 
## Understanding and defining churn
There are two main models of payment in the telecom industry - postpaid (customers pay a monthly/annual bill after using the services) and prepaid (customers pay/recharge with a certain amount in advance and then use the services).
 
In the postpaid model, when customers want to switch to another operator, they usually inform the existing operator to terminate the services, and you directly know that this is an instance of churn.
 
However, in the prepaid model, customers who want to switch to another network can simply stop using the services without any notice, and it is hard to know whether someone has actually churned or is simply not using the services temporarily (e.g. someone may be on a trip abroad for a month or two and then intend to resume using the services again).
 
Thus, churn prediction is usually more critical (and non-trivial) for prepaid customers, and the term ‘churn’ should be defined carefully.  Also, prepaid is the most common model in India and Southeast Asia, while postpaid is more common in Europe in North America.
 
This project is based on the Indian and Southeast Asian market.
 
## Definitions of churn

There are various ways to define churn, such as:

**Revenue-based churn:** Customers who have not utilized any revenue-generating facilities such as mobile internet, outgoing calls, SMS etc. over a given period of time. One could also use aggregate metrics such as ‘customers who have generated less than INR 4 per month in total/average/median revenue’.
 
The main shortcoming of this definition is that there are customers who only receive calls/SMSes from their wage-earning counterparts, i.e. they don’t generate revenue but use the services. For example, many users in rural areas only receive calls from their wage-earning siblings in urban areas.
 
**Usage-based churn:** Customers who have not done any usage, either incoming or outgoing - in terms of calls, internet etc. over a period of time.
 
A potential shortcoming of this definition is that when the customer has stopped using the services for a while, it may be too late to take any corrective actions to retain them. For e.g., if you define churn based on a <u>‘two-months zero usage’</u> period, predicting churn could be useless since by that time the customer would have already switched to another operator.
 
<font color="blue">**In this project, you will use the <u>usage-based definition</u> to define churn.**</font>
 
## High-value churn

In the Indian and the Southeast Asian market, approximately 80% of revenue comes from the top 20% customers (called high-value customers). Thus, if we can reduce churn of the high-value customers, we will be able to reduce significant revenue leakage.
 
In this project, you will define high-value customers based on a certain metric (mentioned later below) and predict churn only on high-value customers.
 
##  Understanding the business objective and the data
The dataset contains customer-level information for a span of four consecutive months - **June, July, August and September**. The months are **encoded** as **6, 7, 8 and 9**, respectively. 

<font color="tomato">The business objective is to **predict the churn in the last (i.e. the ninth) month using** the data (features) from the **first three months**. To do this task well, understanding the typical customer behavior during churn will be helpful.</font>
 
##  Understanding customer behavior during churn

Customers usually do not decide to switch to another competitor instantly, but rather over a period of time (this is especially applicable to high-value customers). In churn prediction, we assume that there are three phases of customer lifecycle :

1.	The **‘good’ phase:** In this phase, the customer is happy with the service and behaves as usual.
2.	The **‘action’ phase:** The customer experience starts to sore in this phase, for e.g. he/she gets a compelling offer from a  competitor, faces unjust charges, becomes unhappy with service quality etc. In this phase, the customer usually shows different behavior than the ‘good’ months. Also, it is crucial to identify high-churn-risk customers in this phase, since some corrective actions can be taken at this point (such as matching the competitor’s offer/improving the service quality etc.)
3.	The **‘churn’ phase:** In this phase, the customer is said to have churned. You define churn based on this phase. Also, it is important to note that at the time of prediction (i.e. the action months), this data is not available to you for prediction. Thus, after tagging churn as 1/0 based on this phase, you discard all data corresponding to this phase.
 
In this case, since you are working over a four-month window, the <u>first two months are the ‘good’ phase</u>, the <u>third month is the ‘action’ phase</u>, while the <u>fourth month is the ‘churn’ phase</u>.


##  Data Preparation
The following data preparation steps are crucial for this problem:
 
1. Derive new features
This is one of the most important parts of data preparation since good features are often the differentiators between good and bad models. Use your business understanding to derive features you think could be important indicators of churn.
 
2. Filter high-value customers
As mentioned above, you need to **predict churn only for the high-value customers**. Define **high-value customers as follows:** Those who have **recharged** with an **amount more than or equal to X, where X is the 70th percentile** of the average recharge amount in the **first two months** (the good phase).
 
After filtering the **high-value customers, you should get about 29.9k rows.**
 
3. Tag churners and remove attributes of the churn phase
Now tag the churned customers (churn=1, else 0) based on the fourth month as follows: Those who have not made any calls (either incoming or outgoing) AND have not used mobile internet even once in the churn phase. The attributes you need to use to tag churners are:

*	total_ic_mou_9
*	total_og_mou_9
*	vol_2g_mb_9
*	vol_3g_mb_9

After tagging churners, remove all the attributes corresponding to the churn phase (all attributes having ‘ _9’, etc. in their names).
 
##  Modelling

<font color="tomato">Build models to predict churn</font>. The predictive model that you’re going to build will serve two purposes:

1.	It will be used to predict whether a high-value customer will churn or not, in near future (i.e. churn phase). By knowing this, the company can take action steps such as providing special plans, discounts on recharge etc.
2.	It will be used to identify important variables that are strong predictors of churn. These variables may also indicate why customers choose to switch to other networks.
 
In some cases, both of the above-stated goals can be achieved by a single machine learning model. But here, you have a large number of attributes, and thus you should try using a dimensionality reduction technique such as PCA and then build a predictive model. After PCA, you can use any classification model.
 
Also, since the rate of churn is typically low (about 5-10%, this is called class-imbalance) - try using techniques to handle class imbalance. 
 
You can take the following suggestive steps to build the model:

1.	Preprocess data (convert columns to appropriate formats, handle **missing values**, etc.)
2.	Conduct appropriate **exploratory analysis** to extract useful insights (whether directly useful for business or for eventual modelling/feature engineering).
3.	Derive **new features**.
4.	Reduce the number of variables using **PCA**.
5.	Train a variety of models, tune model hyperparameters, etc. (**handle class imbalance** using appropriate techniques).
6.	Evaluate the models using appropriate evaluation metrics. Note that it is more important **to identify churners than the non-churners accurately** - choose an appropriate evaluation metric which reflects this business goal. (We need to reduce False-Negative reduce)
7.	Finally, choose a model based on some evaluation metric.
 
The above model will only be able to achieve one of the two goals - 

* to predict customers who will churn. You can’t use the above model 
* to identify the important features for churn. That’s because PCA usually creates components which are not easy to interpret.
 
Therefore, <font color="tomato">**build another model**</font> with the main objective of **identifying important predictor attributes** which help the business understand indicators of churn. A good choice to identify important variables is a logistic regression model or a model from the tree family. In case of logistic regression, make sure to handle multi-collinearity.
 
After identifying important predictors, display them visually - you can use plots, summary tables etc. - whatever you think best conveys the importance of features.
 
Finally, **recommend strategies to manage customer churn** based on your observations.
 
> Note: Everything has to be submitted in one Jupyter notebook.

<hr/>

**OUR UNDERSTANDING:**

> <br/>**\#. Problem Statement**<br/>
> - There are two types of customers:
>    1. Prepaid
>    1. Postpaid
> - We shall focus on `Prepaid` customers
> - There are mainly two types of churning for `Prepaid` customers:
>    1. Revenue-based churn
>    1. Usage-based churn
> - We have upto 20% `High-Value Customers`, who generates 80% revenue.
> - There are 3 phases of churning:
>    1. Good: month 6 and month 7
>    1. Action: month 8
>    1. Churn:  month 9
> - The data is not labled with `Churn:1` or `Not-Churn-0`
> - We need to identify `High-Value Customers` and protect them from `churning` based on `Usage-based churn` technique.
> - After identifying the high-value customers we should flag them as 0 or 1 based on past and current usages stastics
> - We will compare first two month data with 3rd month data and define churn/not-churn
>
> <br/>**\#. Identify High-Value Customers**<br/>
> We need to filter out `high value customers` with the given terms. For high-value customers, we should get about 29.9k rows.
>
> <br/>**\#. Tag Churn:1 or Not-Churn:0**<br/>
> Those who 
> 1. Have not made any calls (either incoming or outgoing): Features to use:  `total_ic_mou_9` & `total_og_mou_9`
> 1. have not used mobile internet even once in the churn phase. Features to use: `vol_2g_mb_9` & `vol_3g_mb_9`
> 
> <br/>**\#. Prohibited Features**<br/>
> We should not use features ended with `_9` or last month
>
> <br/>**\#. Tasks**<br/>
> 1. Build model-1 for prediction using PCA and choose any model to get higher performance as per the business need.
>    - Note: It is more important to identify churners than the non-churners accurately
> 1. Build model-2 to identify best features. No PCA. Only Linear model and Tree Based Models.
> 1. Recommend bussriness strategies

**Custom Functions**

In [6]:
%run ./common.ipynb

# Understaing the data

In [7]:
# load data dictionary
dd = pd.read_excel("Data+Dictionary-+Telecom+Churn+Case+Study.xlsx")
dd

,Acronyms,Descriptions
0,MOBILE_NUMBER,Customer phone number
1,CIRCLE_ID,Telecom circle area to which the customer belongs to
2,LOC,Local calls - within same telecom circle
3,STD,STD calls - outside the calling circle
4,IC,Incoming calls
5,OG,Outgoing calls
6,T2T,"Operator T to T, i.e. within same operator (mobile to mobile)"
7,T2M,Operator T to other operator mobile
8,T2O,Operator T to other operator fixed line
9,T2F,Operator T to fixed lines of T


In [8]:
# load data
df = pd.read_csv("telecom_churn_data.csv")
df.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,offnet_mou_7,offnet_mou_8,offnet_mou_9,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_ic_mou_9,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,roam_og_mou_9,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2t_mou_9,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2m_mou_9,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2f_mou_9,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_t2c_mou_9,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,loc_og_mou_9,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2t_mou_9,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2m_mou_9,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2f_mou_9,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_t2c_mou_9,std_og_mou_6,std_og_mou_7,std_og_mou_8,std_og_mou_9,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,isd_og_mou_9,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,spl_og_mou_9,og_others_6,og_others_7,og_others_8,og_others_9,total_og_mou_6,total_og_mou_7,total_og_mou_8,total_og_mou_9,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2t_mou_9,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2m_mou_9,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_t2f_mou_9,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,loc_ic_mou_9,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2t_mou_9,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2m_mou_9,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2f_mou_9,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_t2o_mou_9,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,std_ic_mou_9,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,total_ic_mou_9,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,spl_ic_mou_9,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,isd_ic_mou_9,ic_others_6,ic_others_7,ic_others_8,ic_others_9,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_num_9,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,total_rech_amt_9,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,max_rech_amt_9,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,last_day_rch_amt_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9,total_rech_data_6,total_rech_data_7,total_rech_data_8,total_rech_data_9,max_rech_data_6,max_rech_data_7,max_rech_data_8,max_rech_data_9,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_2g_9,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,count_rech_3g_9,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,av_rech_amt_data_9,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_2g_mb_9,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,vol_3g_mb_9,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_3g_9,arpu_2g_6,arpu_2g_7,arpu_2g_8,arpu_2g_9,night_pck_user_6,night_pck_user_7,night_pck_user_8,night_pck_user_9,monthly_2g_6,monthly_2g_7,monthly_2g_8,monthly_2g_9,sachet_2g_6,sachet_2g_7,sachet_2g_8,sachet_2g_9,monthly_3g_6,monthly_3g_7,monthly_3g_8,monthly_3g_9,sachet_3g_6,sachet_3g_7,sachet_3g_8,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,214.816,213.803,21.100,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,0.00,0.00,0.00,0.00,NaN,NaN,0.16,NaN,NaN,NaN,4.13,NaN,NaN,NaN,1.15,NaN,NaN,NaN,5.44,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00,0.00,5.44,0.00,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,4,3,

In [9]:
df.shape

(99999, 226)

In [10]:
df = df.drop_duplicates()
df.shape

(99999, 226)

In [11]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99999 entries, 0 to 99998
Data columns (total 226 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   mobile_number             int64  
 1   circle_id                 int64  
 2   loc_og_t2o_mou            float64
 3   std_og_t2o_mou            float64
 4   loc_ic_t2o_mou            float64
 5   last_date_of_month_6      object 
 6   last_date_of_month_7      object 
 7   last_date_of_month_8      object 
 8   last_date_of_month_9      object 
 9   arpu_6                    float64
 10  arpu_7                    float64
 11  arpu_8                    float64
 12  arpu_9                    float64
 13  onnet_mou_6               float64
 14  onnet_mou_7               float64
 15  onnet_mou_8               float64
 16  onnet_mou_9               float64
 17  offnet_mou_6              float64
 18  offnet_mou_7              float64
 19  offnet_mou_8              float64
 20  offnet_mou_9              f

In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
mobile_number,99999.0,7.001207e+09,695669.386290,7.000000e+09,7.000606e+09,7.001205e+09,7.001812e+09,7.002411e+09
circle_id,99999.0,1.090000e+02,0.000000,1.090000e+02,1.090000e+02,1.090000e+02,1.090000e+02,1.090000e+02
loc_og_t2o_mou,98981.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
std_og_t2o_mou,98981.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
loc_ic_t2o_mou,98981.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
arpu_6,99999.0,2.829874e+02,328.439770,-2.258709e+03,9.341150e+01,1.977040e+02,3.710600e+02,2.773109e+04
arpu_7,99999.0,2.785366e+02,338.156291,-2.014045e+03,8.698050e+01,1.916400e+02,3.653445e+02,3.514583e+04
arpu_8,99999.0,2.791547e+02,344.474791,-9.458080e+02,8.412600e+01,1.920800e+02,3.693705e+02,3.354362e+04
arpu_9,99999.0,2.616451e+02,341.998630,-1.899505e+03,6.268500e+01,1.768490e+02,3.534665e+02,3.880562e+04
onnet_mou_6,96062.0,1.323959e+02,297.207406,0.000000e+00,7.380000e+00,3.431000e+01,1.187400e+02,7.376710e+03


In [13]:
# all numeric columns
df.describe().columns.values

array(['mobile_number', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou',
       'loc_ic_t2o_mou', 'arpu_6', 'arpu_7', 'arpu_8', 'arpu_9',
       'onnet_mou_6', 'onnet_mou_7', 'onnet_mou_8', 'onnet_mou_9',
       'offnet_mou_6', 'offnet_mou_7', 'offnet_mou_8', 'offnet_mou_9',
       'roam_ic_mou_6', 'roam_ic_mou_7', 'roam_ic_mou_8', 'roam_ic_mou_9',
       'roam_og_mou_6', 'roam_og_mou_7', 'roam_og_mou_8', 'roam_og_mou_9',
       'loc_og_t2t_mou_6', 'loc_og_t2t_mou_7', 'loc_og_t2t_mou_8',
       'loc_og_t2t_mou_9', 'loc_og_t2m_mou_6', 'loc_og_t2m_mou_7',
       'loc_og_t2m_mou_8', 'loc_og_t2m_mou_9', 'loc_og_t2f_mou_6',
       'loc_og_t2f_mou_7', 'loc_og_t2f_mou_8', 'loc_og_t2f_mou_9',
       'loc_og_t2c_mou_6', 'loc_og_t2c_mou_7', 'loc_og_t2c_mou_8',
       'loc_og_t2c_mou_9', 'loc_og_mou_6', 'loc_og_mou_7', 'loc_og_mou_8',
       'loc_og_mou_9', 'std_og_t2t_mou_6', 'std_og_t2t_mou_7',
       'std_og_t2t_mou_8', 'std_og_t2t_mou_9', 'std_og_t2m_mou_6',
       'std_og_t2m_mou_7', 'std_og

## Datatype Treatment

In [14]:
cat_cols = ['circle_id', 'mobile_number']
df.loc[:, cat_cols] = df.loc[:, cat_cols].astype("category")

In [15]:
# unique mobile numbers
len(df['mobile_number'].unique())

99999

In [16]:
# all RECH features
col_recharges = col_filter(r"_rech_", df)
col_recharges

Index(['total_rech_num_6', 'total_rech_num_7', 'total_rech_num_8',
       'total_rech_num_9', 'total_rech_amt_6', 'total_rech_amt_7',
       'total_rech_amt_8', 'total_rech_amt_9', 'max_rech_amt_6',
       'max_rech_amt_7', 'max_rech_amt_8', 'max_rech_amt_9',
       'date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8',
       'date_of_last_rech_9', 'date_of_last_rech_data_6',
       'date_of_last_rech_data_7', 'date_of_last_rech_data_8',
       'date_of_last_rech_data_9', 'total_rech_data_6', 'total_rech_data_7',
       'total_rech_data_8', 'total_rech_data_9', 'max_rech_data_6',
       'max_rech_data_7', 'max_rech_data_8', 'max_rech_data_9',
       'count_rech_2g_6', 'count_rech_2g_7', 'count_rech_2g_8',
       'count_rech_2g_9', 'count_rech_3g_6', 'count_rech_3g_7',
       'count_rech_3g_8', 'count_rech_3g_9', 'av_rech_amt_data_6',
       'av_rech_amt_data_7', 'av_rech_amt_data_8', 'av_rech_amt_data_9'],
      dtype='object')

In [17]:
# look at rotal_reach items
col_rech = col_filter([r"_rech_"], df)

display(df[col_rech].head())
# unique values
unique(df[col_rech])

,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_num_9,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,total_rech_amt_9,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,max_rech_amt_9,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9,total_rech_data_6,total_rech_data_7,total_rech_data_8,total_rech_data_9,max_rech_data_6,max_rech_data_7,max_rech_data_8,max_rech_data_9,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_2g_9,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,count_rech_3g_9,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,av_rech_amt_data_9
0,4,3,2,6,362,252,252,0,252,252,252,0,6/21/2014,7/16/2014,8/8/2014,9/28/2014,6/21/2014,7/16/2014,8/8/2014,NaN,1.0,1.0,1.0,NaN,252.0,252.0,252.0,NaN,0.0,0.0,0.0,NaN,1.0,1.0,1.0,NaN,252.0,252.0,252.0,NaN
1,4,9,11,5,74,384,283,121,44,154,65,50,6/29/2014,7/31/2014,8/28/2014,9/30/2014,NaN,7/25/2014,8/10/2014,NaN,NaN,1.0,2.0,NaN,NaN,154.0,25.0,NaN,NaN,1.0,2.0,NaN,NaN,0.0,0.0,NaN,NaN,154.0,50.0,NaN
2,5,4,2,7,168,315,116,358,86,200,86,100,6/17/2014,7/24/2014,8/14/2014,9/29/2014,NaN,NaN,NaN,9/17/2014,NaN,NaN,NaN,1.0,NaN,NaN,NaN,46.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,46.0
3,10,11,18,14,230,310,601,410,60,50,50,50,6/28/2014,7/31/2014,8/31/2014,9/30/2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,6,3,4,196,350,287,200,56,110,110,50,6/26/2014,7/28/2014,8/9/2014,9/28/2014,6/4/2014,NaN,NaN,NaN,1.0,NaN,NaN,NaN,56.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,56.0,NaN,NaN,NaN


Unique values: 
total_rech_num_6 : 102
-=-=-=-=-=
total_rech_num_7 : 101
-=-=-=-=-=
total_rech_num_8 : 96
-=-=-=-=-=
total_rech_num_9 : 97
-=-=-=-=-=
total_rech_amt_6 : 2305
-=-=-=-=-=
total_rech_amt_7 : 2329
-=-=-=-=-=
total_rech_amt_8 : 2347
-=-=-=-=-=
total_rech_amt_9 : 2304
-=-=-=-=-=
max_rech_amt_6 : 202
-=-=-=-=-=
max_rech_amt_7 : 183
-=-=-=-=-=
max_rech_amt_8 : 213
-=-=-=-=-=
max_rech_amt_9 : 201
-=-=-=-=-=
date_of_last_rech_6 : 30
-=-=-=-=-=
date_of_last_rech_7 : 31
-=-=-=-=-=
date_of_last_rech_8 : 31
-=-=-=-=-=
date_of_last_rech_9 : 30
-=-=-=-=-=
date_of_last_rech_data_6 : 30
-=-=-=-=-=
date_of_last_rech_data_7 : 31
-=-=-=-=-=
date_of_last_rech_data_8 : 31
-=-=-=-=-=
date_of_last_rech_data_9 : 30
-=-=-=-=-=
total_rech_data_6 : 37
-=-=-=-=-=
total_rech_data_7 : 42
-=-=-=-=-=
total_rech_data_8 : 46
-=-=-=-=-=
total_rech_data_9 : 37
-=-=-=-=-=
max_rech_data_6 : 48
-=-=-=-=-=
max_rech_data_7 : 48
-=-=-=-=-=
max_rech_data_8 : 50
-=-=-=-=-=
max_rech_data_9 : 50
-=-=-=-=-=
count_rech

After looking at the avove features started with `_rech_` we can clearly see, those are:

- Ordinal categorical features 
- Cardinality is very high
- Few features are date. I shall calculate age in days for those columns.
- Few features are numerical

Lets look at one by one:

<!-- I will perform `binning` operation with `frequency-thresholding` on those features:

1. Choose a proportion threshold for a bin.
1. Sort unique values in the column by their frequency in descending order
1. Keep adding the frequency of these sorted (descending) unique values until a threshold is reached.

These are the unique categories we will keep and instances of all other categories shall be replaced by “other”. 

CODE:

# # perform binning with frequency threshold=75%
# for col in col_recharges:
#     transformed_column, _ = frequency_binning(df[col], return_categories_list=True)
#     df.loc[:, col] = transformed_column
#     print(df[col].unique())
# # unique values
# values(df[col_recharges])# # perform binning with frequency threshold=75%
# for col in col_recharges:
#     transformed_column, _ = frequency_binning(df[col], return_categories_list=True)
#     df.loc[:, col] = transformed_column
#     print(df[col].unique())
# # unique values
# values(df[col_recharges])

-->


In [18]:
# look at rotal_reach items
col_rech_dt = col_filter([r"date_of.*rech_"], df)
display(df[col_rech_dt].head())

,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9
0,6/21/2014,7/16/2014,8/8/2014,9/28/2014,6/21/2014,7/16/2014,8/8/2014,NaN
1,6/29/2014,7/31/2014,8/28/2014,9/30/2014,NaN,7/25/2014,8/10/2014,NaN
2,6/17/2014,7/24/2014,8/14/2014,9/29/2014,NaN,NaN,NaN,9/17/2014
3,6/28/2014,7/31/2014,8/31/2014,9/30/2014,NaN,NaN,NaN,NaN
4,6/26/2014,7/28/2014,8/9/2014,9/28/2014,6/4/2014,NaN,NaN,NaN


**Convert to datetime and calculate age of last rechange**

In [19]:
# convert to datetime
from datetime import datetime, date

# This function converts given date to age
def age(born, today):
    delta = today - born
    return delta.days

for col in col_rech_dt:
    df[col] = pd.to_datetime(df[col], format="%m/%d/%Y")

In [20]:
# df[['date_of_last_rech_6_age_day', 
# 'date_of_last_rech_7_age_day', 
# 'date_of_last_rech_8_age_day', 
# 'date_of_last_rech_9_age_day', 
# 'date_of_last_rech_data_6_age_day', 
# 'date_of_last_rech_data_7_age_day', 
# 'date_of_last_rech_data_8_age_day', 
# 'date_of_last_rech_data_9_age_day']].head()

**Lets look at other `date_` features**

In [21]:
ob = df.select_dtypes(include='object')
ob.head()

,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9
0,6/30/2014,7/31/2014,8/31/2014,9/30/2014
1,6/30/2014,7/31/2014,8/31/2014,9/30/2014
2,6/30/2014,7/31/2014,8/31/2014,9/30/2014
3,6/30/2014,7/31/2014,8/31/2014,9/30/2014
4,6/30/2014,7/31/2014,8/31/2014,9/30/2014


In [22]:
# convert above columsn to datetime and calculate age in days
for col in ob.columns:
    df[col] = pd.to_datetime(df[col], format="%m/%d/%Y")

In [23]:
# df[['last_date_of_month_6_age_day', 
# 'last_date_of_month_7_age_day', 
# 'last_date_of_month_8_age_day', 
# 'last_date_of_month_9_age_day']].sum()

**Lets look at the following fetures `sachet_3g_7	sachet_3g_8	sachet_3g_9	fb_user_6	fb_user_7	fb_user_8	fb_user_9`**

In [24]:
col_spl = ['sachet_3g_7','sachet_3g_8','sachet_3g_9',
           'night_pck_user_6','night_pck_user_7','night_pck_user_8','night_pck_user_9',
           'fb_user_6','fb_user_7','fb_user_8','fb_user_9']
values(df[col_spl])

Percent value counts(%): 


0     95.869959
1     2.810028 
2     0.635006 
3     0.229002 
4     0.129001 
5     0.102001 
6     0.064001 
7     0.034000 
8     0.020000 
9     0.020000 
10    0.013000 
11    0.014000 
12    0.013000 
13    0.006000 
14    0.007000 
15    0.007000 
16    0.002000 
17    0.003000 
18    0.005000 
19    0.004000 
20    0.003000 
21    0.002000 
22    0.003000 
24    0.002000 
31    0.001000 
33    0.001000 
35    0.001000 
Name: sachet_3g_7, dtype: float64

-=-=-=-=-=


0     95.809958
1     2.784028 
2     0.638006 
3     0.272003 
4     0.143001 
5     0.105001 
6     0.057001 
7     0.045000 
8     0.033000 
9     0.021000 
10    0.017000 
11    0.010000 
12    0.014000 
13    0.012000 
14    0.007000 
15    0.007000 
16    0.004000 
17    0.002000 
18    0.003000 
19    0.001000 
20    0.003000 
21    0.003000 
23    0.002000 
25    0.001000 
27    0.001000 
29    0.001000 
30    0.002000 
38    0.001000 
41    0.001000 
Name: sachet_3g_8, dtype: float64

-=-=-=-=-=


...

-=-=-=-=-=


0.0    10.919169
1.0    89.080831
Name: fb_user_8, dtype: float64

-=-=-=-=-=


0.0    13.903248
1.0    86.096752
Name: fb_user_9, dtype: float64

-=-=-=-=-=


- Fetures started with `fb_user` & `night_pck_user` are binary columns
- Rest of them are ordinal categorical

In [25]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99999 entries, 0 to 99998
Data columns (total 226 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   mobile_number             category      
 1   circle_id                 category      
 2   loc_og_t2o_mou            float64       
 3   std_og_t2o_mou            float64       
 4   loc_ic_t2o_mou            float64       
 5   last_date_of_month_6      datetime64[ns]
 6   last_date_of_month_7      datetime64[ns]
 7   last_date_of_month_8      datetime64[ns]
 8   last_date_of_month_9      datetime64[ns]
 9   arpu_6                    float64       
 10  arpu_7                    float64       
 11  arpu_8                    float64       
 12  arpu_9                    float64       
 13  onnet_mou_6               float64       
 14  onnet_mou_7               float64       
 15  onnet_mou_8               float64       
 16  onnet_mou_9               float64       
 17  offnet_mou_

**All datatypes are fixed. Lets look cardinality/variance of the columns**

## Unique Value check and cleanup

In [26]:
# unique value counts
unique(df)

Unique values: 
mobile_number : 99999
-=-=-=-=-=
circle_id : 1
-=-=-=-=-=
loc_og_t2o_mou : 1
-=-=-=-=-=
std_og_t2o_mou : 1
-=-=-=-=-=
loc_ic_t2o_mou : 1
-=-=-=-=-=
last_date_of_month_6 : 1
-=-=-=-=-=
last_date_of_month_7 : 1
-=-=-=-=-=
last_date_of_month_8 : 1
-=-=-=-=-=
last_date_of_month_9 : 1
-=-=-=-=-=
arpu_6 : 85681
-=-=-=-=-=
arpu_7 : 85308
-=-=-=-=-=
arpu_8 : 83615
-=-=-=-=-=
arpu_9 : 79937
-=-=-=-=-=
onnet_mou_6 : 24313
-=-=-=-=-=
onnet_mou_7 : 24336
-=-=-=-=-=
onnet_mou_8 : 24089
-=-=-=-=-=
onnet_mou_9 : 23565
-=-=-=-=-=
offnet_mou_6 : 31140
-=-=-=-=-=
offnet_mou_7 : 31023
-=-=-=-=-=
offnet_mou_8 : 30908
-=-=-=-=-=
offnet_mou_9 : 30077
-=-=-=-=-=
roam_ic_mou_6 : 6512
-=-=-=-=-=
roam_ic_mou_7 : 5230
-=-=-=-=-=
roam_ic_mou_8 : 5315
-=-=-=-=-=
roam_ic_mou_9 : 4827
-=-=-=-=-=
roam_og_mou_6 : 8038
-=-=-=-=-=
roam_og_mou_7 : 6639
-=-=-=-=-=
roam_og_mou_8 : 6504
-=-=-=-=-=
roam_og_mou_9 : 5882
-=-=-=-=-=
loc_og_t2t_mou_6 : 13539
-=-=-=-=-=
loc_og_t2t_mou_7 : 13411
-=-=-=-=-=
loc_og_t

**As we see above there are multiple variables having only 1 unique values.**
I will delete the features with 0 variance

In [27]:
# filter columns with only one values
cols_no_variance = list(filter(lambda col: df[col].nunique() == 1, df.columns))
df = df.drop(columns=cols_no_variance)
df.shape

(99999, 210)

**All datatypes are fixed. Lets look at the missing values**

## High Value Customers

### Mising value treatment

To calculate **`high value customers`** we need the following features:


1. **`total_rech_data_M`**
2. **`av_rech_amt_data_M`**
3. **`total_rech_amt_M`**
4. **`date_of_last_rech_data_M`**
5. **`date_of_last_rech_M`**

Here **`M`** is the month **`6, 7, 8 & 9`**

Let us treat the **`Null`** values for the follwoing features.

In [28]:
col_filter(r"date_of_", df)

Index(['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8',
       'date_of_last_rech_9', 'date_of_last_rech_data_6',
       'date_of_last_rech_data_7', 'date_of_last_rech_data_8',
       'date_of_last_rech_data_9'],
      dtype='object')

In [29]:
# look at the missing values
col_reac = col_filter([
    r'total_rech_amt_\d',
    r'date_of_last_rech_\d'
], df)

col_rech_data = col_filter([
    r'total_rech_data_\d',
    r'av_rech_amt_data_\d',
    r'date_of_last_rech_data_\d',
], df)

display(missings(df, col_reac))
print("-=-==-=--=-")
display(missings(df, col_rech_data))


Missing in percent(%): 


date_of_last_rech_9    4.760048
date_of_last_rech_8    3.622036
date_of_last_rech_7    1.767018
date_of_last_rech_6    1.607016
total_rech_amt_9       0.000000
total_rech_amt_8       0.000000
total_rech_amt_7       0.000000
total_rech_amt_6       0.000000
dtype: float64

-=-==-=--=-
Missing in percent(%): 


av_rech_amt_data_6          74.846748
total_rech_data_6           74.846748
date_of_last_rech_data_6    74.846748
av_rech_amt_data_7          74.428744
total_rech_data_7           74.428744
date_of_last_rech_data_7    74.428744
av_rech_amt_data_9          74.077741
total_rech_data_9           74.077741
date_of_last_rech_data_9    74.077741
av_rech_amt_data_8          73.660737
total_rech_data_8           73.660737
date_of_last_rech_data_8    73.660737
dtype: float64

**Analyse the data-rechange columns for missing values**



In [32]:
# collect data-recharge columns for
for m in [6, 7, 8, 9]:
    temp_df = df[[f'date_of_last_rech_data_{m}',f'total_rech_data_{m}', f'max_rech_data_{m}', f'av_rech_amt_data_{m}']]

    display(temp_df.sample(10, random_state=100))

,date_of_last_rech_data_6,total_rech_data_6,max_rech_data_6,av_rech_amt_data_6
79667,2014-06-20,1.0,198.0,198.0
83841,2014-06-11,4.0,125.0,200.0
57025,NaT,NaN,NaN,NaN
53130,NaT,NaN,NaN,NaN
32612,NaT,NaN,NaN,NaN
89899,2014-06-26,5.0,154.0,377.0
67555,NaT,NaN,NaN,NaN
70321,NaT,NaN,NaN,NaN
20410,2014-06-23,1.0,274.0,274.0
6157,NaT,NaN,NaN,NaN


,date_of_last_rech_data_7,total_rech_data_7,max_rech_data_7,av_rech_amt_data_7
79667,2014-07-20,1.0,198.0,198.0
83841,2014-07-11,2.0,152.0,304.0
57025,NaT,NaN,NaN,NaN
53130,NaT,NaN,NaN,NaN
32612,NaT,NaN,NaN,NaN
89899,2014-07-31,8.0,154.0,315.0
67555,NaT,NaN,NaN,NaN
70321,NaT,NaN,NaN,NaN
20410,NaT,NaN,NaN,NaN
6157,NaT,NaN,NaN,NaN


,date_of_last_rech_data_8,total_rech_data_8,max_rech_data_8,av_rech_amt_data_8
79667,2014-08-17,1.0,198.0,198.0
83841,2014-08-06,1.0,154.0,154.0
57025,NaT,NaN,NaN,NaN
53130,2014-08-05,1.0,177.0,177.0
32612,NaT,NaN,NaN,NaN
89899,2014-08-30,6.0,154.0,269.0
67555,NaT,NaN,NaN,NaN
70321,NaT,NaN,NaN,NaN
20410,NaT,NaN,NaN,NaN
6157,NaT,NaN,NaN,NaN


,date_of_last_rech_data_9,total_rech_data_9,max_rech_data_9,av_rech_amt_data_9
79667,2014-09-14,1.0,198.0,198.0
83841,2014-09-04,2.0,152.0,304.0
57025,NaT,NaN,NaN,NaN
53130,NaT,NaN,NaN,NaN
32612,NaT,NaN,NaN,NaN
89899,2014-09-30,8.0,202.0,800.0
67555,NaT,NaN,NaN,NaN
70321,NaT,NaN,NaN,NaN
20410,2014-09-15,1.0,274.0,274.0
6157,NaT,NaN,NaN,NaN


As we can see above, there is a pattern in the missing values. The null values are not at random.

Below code essentially perform **`row wise null`** check. If all item in a row is null or not null we will always have a set of length 1. Thus we can compute this and aggregate the sum for entire dataset. Finally, we can compare the sum with the dataframe size. If both are equal, indicates the null/non-values values are not at random. Whereever there is a missing date there we should find a missing items. Therefore we can easily imputer the null with 0 for those columns.

In [33]:
# collect data-recharge columns for
for m in [6, 7, 8, 9]:
    temp_df = df[[f'date_of_last_rech_data_{m}',f'total_rech_data_{m}', f'max_rech_data_{m}', f'av_rech_amt_data_{m}']]

    temp_sum = (temp_df.apply(lambda x: len(set(x.isna())) , axis=1) == 1).sum()
    print(f"Missing not at Random. Impute with 0. month-{m}\n--=--=--=" if len(temp_df) == temp_sum else "Missing at random")

Missing not at Random. Impute with 0. month-6
--=--=--=
Missing not at Random. Impute with 0. month-7
--=--=--=
Missing not at Random. Impute with 0. month-8
--=--=--=
Missing not at Random. Impute with 0. month-9
--=--=--=


Above columns are reasonable. User did not recharge internet.

In [34]:
# impute data-recharge columns with 0
for m in [6, 7, 8, 9]:
    cols = [f'total_rech_data_{m}', f'max_rech_data_{m}', f'av_rech_amt_data_{m}']
    df[cols] = df[cols].fillna(0)

### Indentify High Value Customers

In [36]:
# get the gross recharge amount for customers for the 'good' period

# get the internet recharge amount
df['total_rech_amt_data_6'] = df['av_rech_amt_data_6'] * df['total_rech_data_6']
df['total_rech_amt_data_7'] = df['av_rech_amt_data_7'] * df['total_rech_data_7']

# add talktime amout with internet amount
df['gross_rech_amt_6'] = df['total_rech_amt_data_6'] + df['total_rech_amt_6']
df['gross_rech_amt_7'] = df['total_rech_amt_data_7'] + df['total_rech_amt_7']

# Calculating gross average amount
df['gross_mean_amt'] = df[['gross_rech_amt_6', 'gross_rech_amt_7']].mean(axis=1)

# find the cutoff for highvalue customers
threshold = 0.7
cut_off = df['gross_mean_amt'].quantile(threshold)

# Filtering the high-value customers
df2 = df[df['gross_mean_amt'] >= cut_off]

df2.shape

(30001, 215)

**We filtered out the high-value customers**

# TODO

## Missing Value Treatment

In [37]:
# lets look at the missing values
missings(df2)

Missing in percent(%): 


date_of_last_rech_data_9    14.781148
night_pck_user_9            14.781148
count_rech_2g_9             14.781148
count_rech_3g_9             14.781148
arpu_3g_9                   14.781148
arpu_2g_9                   14.781148
fb_user_9                   14.781148
count_rech_2g_8             14.048140
count_rech_3g_8             14.048140
arpu_3g_8                   14.048140
date_of_last_rech_data_8    14.048140
night_pck_user_8            14.048140
arpu_2g_8                   14.048140
fb_user_8                   14.048140
fb_user_6                   13.245132
date_of_last_rech_data_6    13.245132
count_rech_2g_6             13.245132
count_rech_3g_6             13.245132
arpu_3g_6                   13.245132
arpu_2g_6                   13.245132
night_pck_user_6            13.245132
fb_user_7                   12.944129
count_rech_2g_7             12.944129
count_rech_3g_7             12.944129
arpu_3g_7                   12.944129
arpu_2g_7                   12.944129
night_pck_us

**Now we have significantly lower missing values in the new dataset**

,mobile_number,av_rech_amt_data_6,av_rech_amt_data_7
0,7000842753,252.0,252.0
1,7001865778,NaN,154.0
2,7001625959,NaN,NaN
3,7001204172,NaN,NaN
4,7000142493,56.0,NaN


In [ ]:
# calculate average
df["avg_rech_good_phase"] = df[col_avg_rech].mean(axis=1)

df[["avg_rech_good_phase"]].head()

,avg_rech_good_phase
0,252.0
1,154.0
2,NaN
3,NaN
4,56.0


In [ ]:
# 70th Percential average rechanre
avg_70th = df[['avg_rech_good_phase']].describe(percentiles=[.7]).T["70%"]
avg_70th

avg_rech_good_phase    221.0
Name: 70%, dtype: float64

In [ ]:
# # 75th Percential average rechanre
avg_70th[0]

221.0

In [ ]:
# get the average value
avg_val = avg_70th[0]
# filter the customers(Numbers)
df2 = df[df['avg_rech_good_phase'] >= avg_val]
df2.shape, df.shape

((9646, 239), (99999, 239))

## Drop fetures for target month

In [ ]:
# all columns ended for month 9
col_mon9 = col_filter(r".+_9$", df)
col_mon9

Index(['arpu_9', 'onnet_mou_9', 'offnet_mou_9', 'roam_ic_mou_9',
       'roam_og_mou_9', 'loc_og_t2t_mou_9', 'loc_og_t2m_mou_9',
       'loc_og_t2f_mou_9', 'loc_og_t2c_mou_9', 'loc_og_mou_9',
       'std_og_t2t_mou_9', 'std_og_t2m_mou_9', 'std_og_t2f_mou_9',
       'std_og_mou_9', 'isd_og_mou_9', 'spl_og_mou_9', 'og_others_9',
       'total_og_mou_9', 'loc_ic_t2t_mou_9', 'loc_ic_t2m_mou_9',
       'loc_ic_t2f_mou_9', 'loc_ic_mou_9', 'std_ic_t2t_mou_9',
       'std_ic_t2m_mou_9', 'std_ic_t2f_mou_9', 'std_ic_mou_9',
       'total_ic_mou_9', 'spl_ic_mou_9', 'isd_ic_mou_9', 'ic_others_9',
       'total_rech_num_9', 'total_rech_amt_9', 'max_rech_amt_9',
       'date_of_last_rech_9', 'last_day_rch_amt_9', 'date_of_last_rech_data_9',
       'total_rech_data_9', 'max_rech_data_9', 'count_rech_2g_9',
       'count_rech_3g_9', 'av_rech_amt_data_9', 'vol_2g_mb_9', 'vol_3g_mb_9',
       'arpu_3g_9', 'arpu_2g_9', 'night_pck_user_9', 'monthly_2g_9',
       'sachet_2g_9', 'monthly_3g_9', 'sachet_3g_9

In [ ]:
# create new dataframe with the month 9 data to used later
df_mon9 = df[col_mon9]
# clear month 9 data from the main dataftrame
df = df.drop(columns=col_mon9)
df.shape, df_mon9.shape

((99999, 167), (99999, 51))

# EDA

# Modle Building - Churn

## Data Preparation

## Modelling

## Model Assesment

## Summary - Churn

# Modle Building - Best Features

## Data Preparation

## Modelling

## Model Assesment

## Summary - Best Feature

# Summary and Business Decisions